In [ ]:
import subprocess
import threading
import time
import mysql.connector as mysql
import pandas as pd
import warnings
import pickle
import numpy as np

# Specify the file of the model for blackjack strategy
filename = './models/blackjack.pkl'
warnings.filterwarnings('ignore')

In [ ]:
# Load the model for blackjack strategy
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
try:
    # Connect to the database
    conn = mysql.connect(
        host='localhost',
        user='root',
        password='19731005',
        database='schema_blackjack'
    )
    cursor = conn.cursor()

    def enqueue_output(process, output_list, detected_classes_list, previous_detected_classes_list):
        
        # Read the output line by line
        for line in iter(process.stdout.readline, b''):
            detected_classes_list.clear()
            # Append the line to the output list
            output_list.append(line.rstrip('\n'))
            # Add the result of the process_output function to the detected_classes_list
            detected_classes_list.extend(process_output(line,previous_detected_classes_list))

        process.stdout.close()

    def process_output(output, previous_detected_classes):
        detected_classes = []
        current_detected_classes = []
        
        # Split the output by comma and space
        result_parts = output.split(", ")
        
        for part in result_parts:
            split_part = part.split(" ")
            # Check if the part has at least 2 elements and the second last element is a digit
            if len(split_part) >= 2 and split_part[-2].isdigit():
                count = int(split_part[-2])
                class_name = split_part[-1]
                
                # Check if the count is equal to 4 detection of card number
                if count in [4]:
                    # Append the class name to the current detected classes
                    current_detected_classes.append(class_name)

        for class_name in current_detected_classes:
            # Check if the class name is not in the detected classes
            if class_name not in detected_classes:
                detected_classes.append(class_name)

        if not previous_detected_classes:
            # Append the current detected classes to the previous detected classes
            previous_detected_classes.extend(current_detected_classes)
            detected_classes = []
        else:
            # If the length is the same between the two lists, it means that the detection is the same and not changed
            if len(previous_detected_classes) == len(current_detected_classes):
                detected_classes = []
            else:
                # Get the last index of the previous detected classes
                index = len(previous_detected_classes)
                # Take the class_name end of previous detected classes for the first index of the current detected classes
                for class_name in current_detected_classes[index:]:
                    # Check if the class name is not in the detected classes
                    if class_name not in detected_classes:
                        detected_classes.append(class_name)
                for class_name in detected_classes:
                    if class_name not in current_detected_classes[index:]:
                        detected_classes.remove(class_name)

        # Clear the previous detected classes and append the current detected classes
        previous_detected_classes.clear()
        previous_detected_classes.extend(current_detected_classes.copy())
        
        # Clear the current detected classes
        results_detection = detected_classes.copy()
        detected_classes.clear()
        current_detected_classes.clear()

        return results_detection

    # Run the script for player and dealer
    def run_script(script_name, source):
        # Run the script with the source of camera
        process = subprocess.Popen(["python", script_name, source], stdout=subprocess.PIPE, text=True)
        
        # Initialize the list needed for the detection
        output_list = []
        detected_classes_list = []
        previous_detected_classes_list = []
        
        # Start the thread for the detection using enqueue_output function
        t = threading.Thread(target=enqueue_output, args=(process, output_list, detected_classes_list, previous_detected_classes_list))
        
        # Set the thread as daemon
        t.daemon = True
        t.start()
        return process, output_list, detected_classes_list, previous_detected_classes_list

    # Run the player and dealer scripts
    player_process, player_output_list, player_detected_classes_list, player_previous_detected_classes_list = run_script("player.py", "0") # 7
    dealer_process, dealer_output_list, dealer_detected_classes_list, dealer_previous_detected_classes_list = run_script("dealer.py", "5") # 12

    # Initialize the variables needed for the detection
    player_previous_output = None
    dealer_previous_output = None

    player_card = None
    dealer_card = None

    detected_player = False
    detected_dealer = False

    print("Starting the detection...")
    
    # Loop until the player and dealer cards are detected
    while True:
        time.sleep(1)

        # Check if the player and dealer processes are terminated
        if player_process.poll() is not None and dealer_process.poll() is not None:
            break

        # Check if the player and dealer output lists are not empty and the detected classes are not the same as the previous detected classes
        if player_output_list and player_detected_classes_list != player_previous_detected_classes_list and detected_player == False:
            # Check if the player has 2 cards
            if len(player_detected_classes_list) == 2:
                print("Player card detected!")
                detected_player = True
                
            previous_output1 = player_detected_classes_list.copy()
            player_output_list.clear()
            
            # Join the detected classes with comma
            player_card = ', '.join(map(str, player_detected_classes_list))

        if dealer_output_list and dealer_detected_classes_list != dealer_previous_detected_classes_list and detected_dealer == False:
            # Check if the dealer has 1 card
            if len(dealer_detected_classes_list) == 1:
                print("Dealer card detected!")
                detected_dealer = True
            previous_output2 = dealer_detected_classes_list.copy()
            dealer_output_list.clear()

            dealer_card = ', '.join(map(str, dealer_detected_classes_list))

        # Check if the player and dealer cards are detected correctly
        if player_card is not None and dealer_card is not None and ',' in player_card and dealer_card != '' and player_card != '':
            
            # Insert the player and dealer cards to the database
            query = f"INSERT INTO classes (player, dealer) VALUES ('{player_card}', '{dealer_card}')"
            
            cursor.execute(query)
            conn.commit()
            
            player_card = None
            dealer_card = None
        
            # Remove all unnecessary rows from the database
            cursor.execute("DELETE FROM classes WHERE player IS NULL OR player = ''")
            cursor.execute("DELETE FROM classes WHERE dealer IS NULL OR dealer = ''")

            # Get the last row from the database to get the last detected player and dealer cards
            query = "SELECT player, dealer FROM classes ORDER BY id DESC LIMIT 1"
            cursor.execute(query)
            player_cards, dealer_cards = cursor.fetchone()

            # See if the number of cards in last detection are correct
            if len(player_cards[0].split(',')) == 2 and len(dealer_cards[0].split(',')) == 1 and player_cards[0] != '' and dealer_cards[0] != '':
                print("Actualizing the database...")
                time.sleep(2)
                break
                
        conn.commit()
        
except mysql.errors.InterfaceError as e:
    error_code = e.errno
    if error_code == 2006:
        print("The MySQL server has gone away. Trying to reconnect...")

        conn = mysql.connect(
            host='localhost',
            user='root',
            password='19731005',
            database='schema_blackjack'
        )

        cursor = conn.cursor()
        conn.commit()
    else:
        raise
finally:
    cursor.close()
    conn.close()

In [ ]:
# Calculate the quality of the game
def calculate_run_count(row):
    # Take firstly the player cards and then the dealer card
    card_list = [row[1], row[2], row[3]]
    run_count = 0
    for card in card_list:
        if card in [2, 3, 4, 5, 6]:
            run_count += 1
        elif card in [10, 'J', 'Q', 'K', 'A']:
            run_count -= 1
    return run_count

# Calculate the quality of the game link to the deck count
def calculate_true_count(row):
    if row[0] != 0:
        deck_count = row[0] / 52
        true_count = row[1] / deck_count
    else:
        true_count = 0
    return true_count

In [ ]:
def get_card_numbers(card_list):
    number_list = []
    
    # Get the number of the cards
    for card in card_list:
        number = card[:-2].strip()
        if number.isdigit():
            number_list.append(int(number))
        # For blackjack, the value is 10
        elif number in ['J', 'Q', 'K']:
            number_list.append(10)
        # For ace, the value is 1 or 11
        elif number == 'A':
            number_list.append(11)
    return numbers

prediction = None

try:
    conn = mysql.connect(
        host='localhost',
        user='root',
        password='19731005',
        database='schema_blackjack'
    )

    cursor = conn.cursor()

    # Get the last detected player and dealer cards
    query = "SELECT CONCAT(player, ', ', dealer) as combined FROM classes ORDER BY id DESC LIMIT 1"
    cursor.execute(query)
    result = cursor.fetchone()
    
    cards = result[0].split(', ')
    numbers = get_card_numbers(cards)
    print(numbers)
       
    conn.commit()

    # Get copy of the numbers
    prediction = numbers.copy()
    
    #TODO: Implement the blackjack detection and deck count
    
    # Insert value for deck count
    prediction.insert(0, 178)
    
    # Insert value for run and true count
    prediction.insert(1, calculate_run_count(prediction))
    prediction.insert(2, calculate_true_count(prediction))
    
    # If the dealer has blackjack
    prediction.insert(-1, 0)
    
    # If the player has blackjack
    prediction.insert(-2, 0)
except mysql.errors.InterfaceError as e:
    error_code = e.errno
    if error_code == 2006:
        print("The MySQL server has gone away. Trying to reconnect...")

        conn = mysql.connect(
            host='localhost',
            user='root',
            password='19731005',
            database='schema_blackjack'
        )

        cursor = conn.cursor()
        conn.commit()
    else:
        raise
finally:
    cursor.close()
    conn.close()

In [ ]:
# Define the feature names
feature_names = ['cards_remaining','run_count', 'true_count', 'player_card_1', 'player_card_2', 'dealer_card_1', 'is_blackjack_dealer', 'is_blackjack_player']

# Define the action dictionary
action_dict = {0: 'No Action', 1: 'Stand', 2: 'Hit', 3: 'Double', 4: 'Split', 5: 'Surrender', 6: 'No Insurance'}

In [ ]:
#TODO: Implement the interface for the action

if prediction is not None:
    # Reshape the prediction
    prediction_array = np.array(prediction)
    prediction_reshaped = prediction_array.reshape(1, -1)
    prediction_df = pd.DataFrame(prediction_reshaped, columns=feature_names)
    
    # Predict the strategy
    strategy = loaded_model.predict(prediction_df)

    # Get the action name of the strategy
    action_name = [action_dict[i] for i in strategy[0] if i != 0]
    
    # Print the action
    print(f"The action is: {action_name}")